# 0. 환경설정 및 워크벤치 세팅

## 워크벤치 세팅

In [ ]:
! pip install pandas sqlalchemy pymysql -q

In [ ]:
user = 'root'
password = 'password'
host = 'localhost'
port = 3306
database = 'Kaggle_HR' 

In [ ]:
from sqlalchemy import create_engine

# 1) 커넥션 URL 생성
url = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}?charset=utf8mb4"
engine = create_engine(url)

In [ ]:
data_path = 'wfs_behaviors_and_records_508p-546d-98r_20220722173739.csv'
hr_df = pd.read_csv(data_path, encoding='cp1252')

In [ ]:
hr_df.to_sql('hr_table', engine, if_exists='append', index=False)

## VSCODE 세팅

In [ ]:
! pip install koreanize_matplotlib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

In [ ]:
data_path = 'wfs_behaviors_and_records_508p-546d-98r_20220722173739.csv'
hr_df = pd.read_csv(data_path, encoding='cp1252')

# 1. 우리 회사 직원 현황

In [31]:
hr_df.head(2)

,sub_ID,sub_fname,sub_lname,sub_age,sub_sex,sub_shift,sub_team,sub_role,sub_coll_IDs,sub_colls_same_sex_prtn,...,event_weekday_num,event_weekday_name,behav_comptype_h,behav_cause_h,actual_efficacy_h,record_comptype,record_cause,recorded_efficacy,recorded_note_from_sup,record_conf_matrix_h
0,98000001,Rebecca,Bauer,40,F,Shift 1,Team 1,Team Leader,"[98000002, 98000003, 98000004, 98000005, 98000...",0.714286,...,4,Friday,Presence,NaN,NaN,Presence,NaN,NaN,NaN,NaN
1,98000001,Rebecca,Bauer,40,F,Shift 1,Team 1,Team Leader,"[98000002, 98000003, 98000004, 98000005, 98000...",0.714286,...,4,Friday,Efficacy,NaN,1.489,Efficacy,NaN,1.2,NaN,NaN


In [ ]:
# 직원 기초 현황
unique_df = hr_df.drop_duplicates(subset='sub_ID')

total_count = unique_df['sub_ID'].nunique()
avg_age = unique_df['sub_age'].mean()
total_female = (unique_df['sub_sex'] == 'F').sum()  # () 조건이 참인 경우 1
total_male = (unique_df['sub_sex'] == 'M').sum()

print(f'전체 인원: {total_count}명')
print(f'나이 평균: {avg_age:.1f}')
print(f'여성 직원 수: {total_female}명')
print(f'남성 직원 수: {total_male}명')

전체 인원: 687명
나이 평균: 41.4
여성 직원 수: 359명
남성 직원 수: 328명


In [59]:
hr_df.columns

Index(['sub_ID', 'sub_fname', 'sub_lname', 'sub_age', 'sub_sex', 'sub_shift',
       'sub_team', 'sub_role', 'sub_coll_IDs', 'sub_colls_same_sex_prtn',
       'sub_health_h', 'sub_commitment_h', 'sub_perceptiveness_h',
       'sub_dexterity_h', 'sub_sociality_h', 'sub_goodness_h',
       'sub_strength_h', 'sub_openmindedness_h', 'sub_workstyle_h', 'sup_ID',
       'sup_fname', 'sup_lname', 'sup_age', 'sup_sub_age_diff', 'sup_sex',
       'sup_role', 'sup_commitment_h', 'sup_perceptiveness_h',
       'sup_goodness_h', 'event_date', 'event_week_in_series',
       'event_day_in_series', 'event_weekday_num', 'event_weekday_name',
       'behav_comptype_h', 'behav_cause_h', 'actual_efficacy_h',
       'record_comptype', 'record_cause', 'recorded_efficacy',
       'recorded_note_from_sup', 'record_conf_matrix_h'],
      dtype='object')

In [63]:
# 직원 역량 평균 확인
avg_health = unique_df['sub_health_h'].mean()
avg_commitment = unique_df['sub_commitment_h'].mean()
avg_perceptiveness = unique_df['sub_perceptiveness_h'].mean()
avg_dexterity = unique_df['sub_dexterity_h'].mean()
avg_sociality = unique_df['sub_sociality_h'].mean()
avg_goodness = unique_df['sub_goodness_h'].mean()
avg_strength = unique_df['sub_strength_h'].mean()
avg_openmindness = unique_df['sub_openmindedness_h'].mean()

print('우리 회사의 직원 역량 평균')
print(f'건강: {avg_health:.3f}')
print(f'헌신도: {avg_commitment:.3f}')
print(f'인지력: {avg_perceptiveness:.3f}')
print(f'손재주: {avg_dexterity:.3f}')
print(f'사회성: {avg_sociality:.3f}')
print(f'선량함: {avg_goodness:.3f}')
print(f'힘: {avg_strength:.3f}')
print(f'수용성: {avg_openmindness:.3f}')

우리 회사의 직원 역량 평균
건강: 0.743
헌신도: 0.747
인지력: 0.742
손재주: 0.736
사회성: 0.740
선량함: 0.740
힘: 0.740
수용성: 0.754


In [ ]:
# shift, team 별 현황
shift_info = unique_df.groupby(['sub_shift', 'sub_team']).agg({
	'sub_ID': 'nunique',
	'sub_age' : 'mean',
  	'sub_sex': [
    	('여성', lambda x: (x=='F').sum()),
   		('남성', lambda x: (x=='M').sum())
  ]
})

shift_info.reset_index(inplace=True)
shift_info.columns = ['shift', 'team', '인원수', '평균 나이', '여성', '남성']
print(shift_info)

         shift        team  인원수      평균 나이  여성  남성
0      Shift 1      Team 1   29  37.965517  12  17
1      Shift 1      Team 2   26  45.884615  15  11
2      Shift 1      Team 3   28  40.392857  16  12
3      Shift 1      Team 4   28  38.321429  14  14
4      Shift 1      Team 5   27  41.666667  15  12
5      Shift 1      Team 6   31  41.612903  17  14
6      Shift 1      Team 7   25  40.120000  12  13
7      Shift 1      Team 8   31  45.129032  13  18
8      Shift 1  unassigned    1  24.000000   1   0
9      Shift 2     Team 10   27  42.333333  10  17
10     Shift 2     Team 11   30  43.933333  19  11
11     Shift 2     Team 12   34  45.676471  25   9
12     Shift 2     Team 13   25  39.560000  18   7
13     Shift 2     Team 14   31  39.290323  12  19
14     Shift 2     Team 15   31  37.709677  14  17
15     Shift 2     Team 16   26  41.230769  12  14
16     Shift 2      Team 9   25  40.600000  12  13
17     Shift 2  unassigned    1  23.000000   0   1
18     Shift 3     Team 17   28

In [66]:
# shift, team 별 역량 평균
shift_info = unique_df.groupby(['sub_shift', 'sub_team']).agg({
  'sub_health_h':'mean',
  'sub_commitment_h' :'mean',
  'sub_perceptiveness_h':'mean',
  'sub_dexterity_h':'mean',
  'sub_sociality_h':'mean',
  'sub_goodness_h':'mean',
  'sub_strength_h':'mean',
  'sub_openmindedness_h':'mean'
})

shift_info.reset_index(inplace=True)
shift_info.columns = ['shift', 'team', '건강', '헌신도', '인지력', '손재주', '사회성', '선량함', '힘', '수용성']
print(shift_info)

         shift        team        건강       헌신도       인지력       손재주       사회성  \
0      Shift 1      Team 1  0.706310  0.757172  0.714207  0.739103  0.783621   
1      Shift 1      Team 2  0.727346  0.660654  0.674038  0.745115  0.783962   
2      Shift 1      Team 3  0.756571  0.757321  0.760000  0.783643  0.735607   
3      Shift 1      Team 4  0.708107  0.754214  0.783214  0.730643  0.721250   
4      Shift 1      Team 5  0.714148  0.770222  0.783667  0.750630  0.734704   
5      Shift 1      Team 6  0.698871  0.775710  0.722516  0.702323  0.698161   
6      Shift 1      Team 7  0.675120  0.761960  0.738920  0.711280  0.776720   
7      Shift 1      Team 8  0.762032  0.729129  0.746548  0.766097  0.741032   
8      Shift 1  unassigned  0.983000  0.842000  0.948000  0.836000  0.830000   
9      Shift 2     Team 10  0.719259  0.781593  0.764444  0.750185  0.694630   
10     Shift 2     Team 11  0.781733  0.756333  0.728867  0.691733  0.720867   
11     Shift 2     Team 12  0.719029  0.